In [4]:
import os
import subprocess
import json

# 현재 디렉토리에서 모든 하위 디렉토리 가져오기

def commit_logger(project_names):
    output_directory = "commit-logs"

    # 만약 output_directory 디렉토리가 없다면 생성
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    
    for directory in project_names:
        if not os.path.exists(os.path.join(directory, ".git")):
            continue  # Git 저장소가 아닌 경우 스킵

        os.chdir(directory)  # 리포지토0리 디렉토리로 이동

        # 스크립트를 실행하여 Git 로그를 파일로 저장
        start_date = "2021-10-01"
        end_date = "2023-07-18"  # 원하는 날짜로 수정
        output_file = f"{directory}-log.json"

        git_log_command = [
            "git",
            "log",
            f"--since={start_date}",
            f"--until={end_date}",
            "--pretty=format:{%n  ^^^^commitHash^^^^: ^^^^%H^^^^,  %n  ^^^^parent^^^^: ^^^^%P^^^^,  %n  ^^^^subject^^^^: ^^^^%s^^^^,  %n  ^^^^sanitized_subject_line^^^^: ^^^^%f^^^^,  %n  ^^^^commit_notes^^^^: ^^^^%N^^^^,  %n  ^^^^body^^^^: ^^^^%b^^^^,  %n  ^^^^author_name^^^^: ^^^^%aN^^^^,  ^^^^committer_name^^^^: ^^^^%cN^^^^,%n    ^^^^date^^^^: ^^^^%cD^^^^%n},"
            "'| sed 's/"/\\"/g' | sed 's/\^^^^/"/g' | sed "$ s/,$//" | sed -e ':a' -e 'N' -e '$!ba' -e 's/\n/ /g' | tr -d '\r\n' | tr -d '	' | tr -d '	' | awk 'BEGIN { print("[") } { print($0) } END { print("]") }'",
            ">",
            f"{output_file}"
        ]

        # Git 로그를 파일로 저장
        with open(output_file, "w") as log_file:
            subprocess.call(git_log_command, stdout=log_file, universal_newlines=True, shell=True)

        os.chdir("..")  # 리포지토리 디렉토리에서 나감

    # 모든 로그 파일을 읽어와서 JSON 형식으로 변환
    commits = []
    for directory in project_names:
        log_file = f"{directory}-log.json"
        with open(log_file, "r") as f:
            try:
                log_data = json.load(f)
                commits.extend(log_data)
            except json.JSONDecodeError:
                # JSON 파싱 오류가 발생한 경우 줄 단위로 읽어서 이어붙이기
                with open(log_file, "r") as f:
                    lines = f.readlines()
                    combined_lines = ""
                    for line in lines:
                        # "\"를 "\\"로 치환
                        line = line.replace("\\", "\\\\")
                        combined_lines += line.strip()
                        try:
                            log_data = json.loads(combined_lines)
                            commits.extend(log_data)
                            combined_lines = ""
                        except json.JSONDecodeError:
                            continue

    # 최종 commits 리스트에 저장
    output_path = os.path.join(output_directory, "all-commits.json")
    with open(output_path, "w") as f:
        json.dump(commits, f, indent=2)

    print("모든 리포지토리의 Git 로그를 JSON 파일로 저장하고 병합하였습니다.")

if __name__ == "__main__":
    directories = ["bc-java", "pgjdbc", "junit4", "h2database", "gson", "guava"]
    commit_logger(directories)


SyntaxError: unexpected character after line continuation character (473418968.py, line 31)